In [1]:
import pandas as pd
import numpy as np

## Model Training

In [2]:
df = pd.read_csv('eda_cleaned.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Distance,Time_taken_min
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,2022-12-02 21:55:00,...,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,10.29,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,2022-02-13 14:55:00,...,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,6.25,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,2022-04-03 17:30:00,...,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,13.80,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,2022-02-13 09:20:00,...,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,2.93,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,2022-02-14 19:50:00,...,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,19.42,41


In [3]:
df=df.drop(labels=['ID','Delivery_person_ID','Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude','Order_Date','Time_Orderd','Time_Order_picked'],axis=1)

In [4]:
## Independent and dependent features
X = df.drop(labels=['Time_taken_min'],axis=1)
Y = df[['Time_taken_min']]

In [5]:
Y

,Time_taken_min
0,46
1,23
2,21
3,20
4,41
...,...
33778,32
33779,36
33780,16
33781,26


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33783 entries, 0 to 33782
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Delivery_person_Age      33783 non-null  float64
 1   Delivery_person_Ratings  33783 non-null  float64
 2   Prep_time                33783 non-null  float64
 3   Weather_conditions       33783 non-null  object 
 4   Road_traffic_density     33783 non-null  object 
 5   Vehicle_condition        33783 non-null  int64  
 6   Type_of_order            33783 non-null  object 
 7   Type_of_vehicle          33783 non-null  object 
 8   multiple_deliveries      33783 non-null  float64
 9   Festival                 33783 non-null  object 
 10  City                     33783 non-null  object 
 11  Distance                 33783 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 3.1+ MB


In [7]:
# Define which columns should be ordinal-encoded and which should be scaled
Ordinal_cols = ['Road_traffic_density']
Nominal_cols = ['Weather_conditions','Type_of_order','Type_of_vehicle','Festival','City']
numerical_cols = X.select_dtypes(exclude='object').columns

In [8]:
df.multiple_deliveries.unique()

array([3., 1., 0., 2.])

In [9]:
# Define the custom ranking for each ordinal variable
Traffic_categories = ['Low','Medium','High','Jam']
#Delivery_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
#clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.preprocessing import OneHotEncoder #Onehot encoder
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
ord_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[Traffic_categories])),
    ('scaler',StandardScaler())
    ]

)

# Categorigal Pipeline
nom_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehotencoder',OneHotEncoder(sparse=False)),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
#('num_pipeline',num_pipeline,numerical_cols)
('num_pipeline',num_pipeline,numerical_cols),

('ord_pipeline',ord_pipeline,Ordinal_cols),
('nom_pipeline',nom_pipeline,Nominal_cols)
])


In [12]:
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

In [13]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

<h1>###################################################################</h1>

In [14]:
dummy = X[['Delivery_person_Age','Delivery_person_Ratings','Weather_conditions']]

In [15]:
dummy

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions
0,36.0,4.2,Fog
1,21.0,4.7,Stormy
2,23.0,4.7,Sandstorms
3,34.0,4.3,Sandstorms
4,24.0,4.7,Fog
...,...,...,...
33778,30.0,4.8,Windy
33779,21.0,4.6,Windy
33780,30.0,4.9,Cloudy
33781,20.0,4.7,Cloudy


In [16]:
preprocessor.fit_transform(dummy)

ValueError: A given column is not a column of the dataframe

In [ ]:
get_column_names_from_ColumnTransformer(preprocessor)

In [ ]:
en = preprocessor.named_transformers_['nom_pipeline']
en

In [22]:
preprocessor

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('num_pipeline',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0)),
                                                 ('scaler',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                  

In [12]:
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

In [30]:
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        raw_col_name_reverse = raw_col_name[::-1]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
            exchange_name = [(_.split("_")) for _ in preprocessor.transformers_[:-1][0][1].steps[-1][1].get_feature_names()]
            last_pre_name = ""
            last_raw_name = ""
            for pre_name,value in exchange_name:
                if pre_name==last_pre_name:
                    col_name.append(last_raw_name+"_"+value)
                if pre_name!=last_pre_name:
                    last_pre_name=pre_name
                    last_raw_name=raw_col_name_reverse.pop()
                    col_name.append(last_raw_name+"_"+value)
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

In [34]:
import sklearn
import pandas as pd

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [36]:
import warnings
get_feature_names(preprocessor)

C:\Users\ramag\anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Transformer imputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
C:\Users\ramag\anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
C:\Users\ramag\anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Transformer ordinalencoder (type OrdinalEncoder) does not provide get_feature_names. Will return input column names if available


['num_pipeline__Delivery_person_Age',
 'num_pipeline__Delivery_person_Ratings',
 'num_pipeline__Prep_time',
 'num_pipeline__Vehicle_condition',
 'num_pipeline__multiple_deliveries',
 'num_pipeline__Distance',
 'ord_pipeline__Road_traffic_density',
 'onehotencoder__x0_Cloudy',
 'onehotencoder__x0_Fog',
 'onehotencoder__x0_Sandstorms',
 'onehotencoder__x0_Stormy',
 'onehotencoder__x0_Sunny',
 'onehotencoder__x0_Windy',
 'onehotencoder__x1_Buffet',
 'onehotencoder__x1_Drinks',
 'onehotencoder__x1_Meal',
 'onehotencoder__x1_Snack',
 'onehotencoder__x2_electric_scooter',
 'onehotencoder__x2_motorcycle',
 'onehotencoder__x2_scooter',
 'onehotencoder__x3_No',
 'onehotencoder__x3_Yes',
 'onehotencoder__x4_Metropolitian',
 'onehotencoder__x4_Semi-Urban',
 'onehotencoder__x4_Urban']

In [31]:
get_column_names_from_ColumnTransformer(preprocessor)

['Road_traffic_density']

In [28]:
preprocessor.transformers_[:-1]

[('num_pipeline',
  Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='median',
                                 verbose=0)),
                  ('scaler',
                   StandardScaler(copy=True, with_mean=True, with_std=True))],
           verbose=False),
  Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Prep_time',
         'Vehicle_condition', 'multiple_deliveries', 'Distance'],
        dtype='object')),
 ('ord_pipeline',
  Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='most_frequent',
                                 verbose=0)),
                  ('ordinalencoder',
                   OrdinalEncoder(categories=[['Low', 'Medium', 'High', 'Jam']],
                                

In [29]:
for p in preprocessor.transformers_[:-1]:
    print(p[2])

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Prep_time',
       'Vehicle_condition', 'multiple_deliveries', 'Distance'],
      dtype='object')
['Road_traffic_density']


<h1>###################################################################</h1>

In [17]:
X_train.head()

,Delivery_person_Age,Delivery_person_Ratings,Prep_time,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Distance
11324,33.0,4.7,15.0,Sunny,High,2,Drinks,scooter,0.0,No,Urban,3.04
3106,27.0,4.9,15.0,Cloudy,Low,2,Meal,scooter,1.0,No,Metropolitian,19.91
14753,30.0,4.9,5.0,Sandstorms,Jam,0,Drinks,motorcycle,1.0,Yes,Metropolitian,13.99
3594,39.0,4.2,15.0,Cloudy,High,2,Snack,scooter,1.0,No,Metropolitian,6.30
1057,31.0,5.0,15.0,Sunny,Jam,2,Snack,scooter,1.0,No,Metropolitian,20.47


In [18]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train))
X_test=pd.DataFrame(preprocessor.transform(X_test))

#X_train=pd.DataFrame(preprocessor.fit_transform(X_train))
#X_test=pd.DataFrame(preprocessor.transform(X_test))


In [19]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.588912,0.215251,1.230278,1.224845,-1.296163,-0.080171,0.479559,-0.445420,-0.456216,-0.444397,...,-0.580540,-0.58002,-0.297431,-1.194020,1.421737,0.145461,-0.145461,-1.827751,-0.061809,1.847590
1,-0.451913,0.842401,1.230278,1.224845,0.444222,-0.024586,-1.124375,2.245074,-0.456216,-0.444397,...,1.722533,-0.58002,-0.297431,-1.194020,1.421737,0.145461,-0.145461,0.547120,-0.061809,-0.541246
2,0.068499,0.842401,-1.220124,-1.215352,0.444222,-0.044092,1.281526,-0.445420,-0.456216,2.250242,...,-0.580540,-0.58002,-0.297431,0.837507,-0.703365,-6.874680,6.874680,0.547120,-0.061809,-0.541246
3,1.629737,-1.352624,1.230278,1.224845,0.444222,-0.069430,0.479559,2.245074,-0.456216,-0.444397,...,-0.580540,1.72408,-0.297431,-1.194020,1.421737,0.145461,-0.145461,0.547120,-0.061809,-0.541246
4,0.241970,1.155976,1.230278,1.224845,0.444222,-0.022741,1.281526,-0.445420,-0.456216,-0.444397,...,-0.580540,1.72408,-0.297431,-1.194020,1.421737,0.145461,-0.145461,0.547120,-0.061809,-0.541246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23643,1.629737,0.842401,1.230278,0.004747,0.444222,-0.085278,-1.124375,2.245074,-0.456216,-0.444397,...,-0.580540,-0.58002,-0.297431,-1.194020,1.421737,0.145461,-0.145461,-1.827751,-0.061809,1.847590
23644,1.109325,1.155976,-1.220124,-1.215352,2.184607,-0.054339,-0.322408,-0.445420,-0.456216,2.250242,...,-0.580540,1.72408,-0.297431,0.837507,-0.703365,0.145461,-0.145461,0.547120,-0.061809,-0.541246
23645,-1.666210,0.528826,-1.220124,1.224845,0.444222,-0.069693,-0.322408,-0.445420,-0.456216,2.250242,...,-0.580540,-0.58002,3.362120,-1.194020,-0.703365,0.145461,-0.145461,0.547120,-0.061809,-0.541246
23646,-0.278443,1.155976,-1.220124,0.004747,0.444222,-0.054339,-1.124375,-0.445420,-0.456216,2.250242,...,1.722533,-0.58002,-0.297431,0.837507,-0.703365,0.145461,-0.145461,0.547120,-0.061809,-0.541246


In [20]:
get_column_names_from_ColumnTransformer(preprocessor)

['Road_traffic_density']

In [21]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.588912,0.215251,1.230278,1.224845,-1.296163,-0.080171,0.479559,-0.445420,-0.456216,-0.444397,...,-0.580540,-0.58002,-0.297431,-1.194020,1.421737,0.145461,-0.145461,-1.827751,-0.061809,1.847590
1,-0.451913,0.842401,1.230278,1.224845,0.444222,-0.024586,-1.124375,2.245074,-0.456216,-0.444397,...,1.722533,-0.58002,-0.297431,-1.194020,1.421737,0.145461,-0.145461,0.547120,-0.061809,-0.541246
2,0.068499,0.842401,-1.220124,-1.215352,0.444222,-0.044092,1.281526,-0.445420,-0.456216,2.250242,...,-0.580540,-0.58002,-0.297431,0.837507,-0.703365,-6.874680,6.874680,0.547120,-0.061809,-0.541246
3,1.629737,-1.352624,1.230278,1.224845,0.444222,-0.069430,0.479559,2.245074,-0.456216,-0.444397,...,-0.580540,1.72408,-0.297431,-1.194020,1.421737,0.145461,-0.145461,0.547120,-0.061809,-0.541246
4,0.241970,1.155976,1.230278,1.224845,0.444222,-0.022741,1.281526,-0.445420,-0.456216,-0.444397,...,-0.580540,1.72408,-0.297431,-1.194020,1.421737,0.145461,-0.145461,0.547120,-0.061809,-0.541246


In [ ]:
preprocessor.get_feature_names()

In [ ]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
regression=LinearRegression()
regression.fit(X_train,y_train)

In [ ]:
regression.coef_

In [ ]:
regression.intercept_

In [ ]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


In [ ]:
model_list